In [1]:
import asyncio
from mavsdk import System
from dataclasses import dataclass
from mavsdk.telemetry import LandedState

@dataclass
class Position:
    lat: float
    long: float
    alt: float

async def get_position(drone):
    while True:
        global loc
        position = await drone.telemetry.position().__aiter__().__anext__()
        lat = round(position.latitude_deg, 5)
        long = round(position.longitude_deg, 5)
        alt = round(position.absolute_altitude_m, 1)
        loc = Position(lat,long,alt)
        await asyncio.sleep(.1)

async def pos_reached(pos):
    
    lat = round(pos[0], 4)
    long = round(pos[1], 4)
    alt = round(pos[2], 1)
    while True:
        global loc
        lat_reached = round(loc.lat, 4)==lat
        long_reached = round(loc.long, 4)==long
        alt_reached = round(loc.alt, 4)==alt
        if False:
            print("from " + str(loc.lat) + " to " + str(lat) + ": " + str(lat_reached))
            print("from " + str(loc.long) + " to " + str(long) + ": " + str(long_reached))
            print("from " + str(loc.alt) + " to " + str(alt) + ": " + str(alt_reached))
        if lat_reached & long_reached & alt_reached:
            print("Reached location.")
            return
        else:
            await asyncio.sleep(1)

global loc
loc = Position(0,0,0)

drone = System(port=50050)
await drone.connect(system_address="udp://:14540")

print("Waiting for drone to connect...")
async for state in drone.core.connection_state():
    if state.is_connected:
        print(f"-- Connected to drone!")
        break

async for health in drone.telemetry.health():
    if health.is_global_position_ok and health.is_home_position_ok:
        print("-- Global position state is good enough for flying.")
        break

pos = [47.39907273002228, 8.54371614376562, 500]

task = asyncio.create_task(get_position(drone))

print("arming")
await drone.action.arm()
print("takeoff")
await drone.action.takeoff()
print("goto")
await drone.action.goto_location(*pos, 0)
await pos_reached(pos)
print("landing")
await drone.action.land()
while False:
    state = await drone.telemetry.landed_state().__aiter__().__anext__()
    print(state)
    if state == "ON_GROUND":
        print("we landed")
        break
    await asyncio.sleep(0.5)
state = await drone.telemetry.landed_state().__aiter__().__anext__()

task.cancel()



Waiting for drone to connect...
-- Connected to drone!
-- Global position state is good enough for flying.
arming
takeoff
goto
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 488.0 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 488.0 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 488.0 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 488.0 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 488.0 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 488.5 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 489.3 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 490.5 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 493.5 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 496.0 to 500: False
from 47.39907 to 47.399

Exception ignored in: <function WrappedIterator.__del__ at 0x7f176c553880>
Traceback (most recent call last):
  File "/home/surenottoday/.local/lib/python3.10/site-packages/aiogrpc/utils.py", line 168, in __del__
    self._stream_executor.shutdown()
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1093, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 499.6 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 499.8 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 499.8 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 499.9 to 500: False
from 47.39907 to 47.3991: True
from 8.54372 to 8.5437: True
from 500.0 to 500: True
Reached location.
landing


True